# AUTOMATIC DIFFERENTIATION WITH TORCH.AUTOGRAD

In [1]:
import torch

x = torch.ones(5)  # input tensor
y = torch.zeros(3)  # expected output
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w)+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

# Tensors, Functions and Computational graph

![comp-graph](.\picture\comp-graph.png)

In [2]:
print('Gradient function for z =', z.grad_fn)
print('Gradient function for loss =', loss.grad_fn)

Gradient function for z = <AddBackward0 object at 0x000001AA2F519400>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward0 object at 0x000001AA2F519D00>


# Computing Gradients

In [3]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.1760, 0.2868, 0.3322],
        [0.1760, 0.2868, 0.3322],
        [0.1760, 0.2868, 0.3322],
        [0.1760, 0.2868, 0.3322],
        [0.1760, 0.2868, 0.3322]])
tensor([0.1760, 0.2868, 0.3322])


# Disabling Gradient Tracking

In [4]:
z = torch.matmul(x, w)+b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w)+b
print(z.requires_grad)

True
False


下面的代码段可以达到与上面一样的效果

In [5]:
z = torch.matmul(x, w)+b
z_det = z.detach()
print(z_det.requires_grad)

False


# Optional Reading: Tensor Gradients and Jacobian Products

In [6]:
inp = torch.eye(2,3, requires_grad=True)
out = (inp+1).pow(2)
out.backward(torch.ones_like(inp), retain_graph=True)
print("First call\n", inp.grad)
out.backward(torch.ones_like(inp), retain_graph=True)
print("\nSecond call\n", inp.grad)
inp.grad.zero_()
out.backward(torch.ones_like(inp), retain_graph=True)
print("\nCall after zeroing gradients\n", inp.grad)

First call
 tensor([[4., 2., 2.],
        [2., 4., 2.]])

Second call
 tensor([[8., 4., 4.],
        [4., 8., 4.]])

Call after zeroing gradients
 tensor([[4., 2., 2.],
        [2., 4., 2.]])


上述过程与下述过程是等价的

In [12]:
inp = torch.eye(2,3, requires_grad=True)
out = (inp+1).pow(2)
out.sum().backward(retain_graph=True)
print("First call\n", inp.grad)
out.sum().backward(retain_graph=True)
print("\nSecond call\n", inp.grad)
inp.grad.zero_()
out.sum().backward(retain_graph=True)
print("\nCall after zeroing gradients\n", inp.grad)

First call
 tensor([[4., 2., 2.],
        [2., 4., 2.]])


RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.